# Analysis of IPL DATA

"Sports Basics" is a sports blog company that entered space recently.  They wanted to get more traffic to their website by releasing a special edition magazine on IPL 2024.  This magazine aims to provide interesting insights and facts for fans, analysts and teams based on the last 3 years' data.

### Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Importing Data

In [2]:
match_summary = pd.read_csv('D:\Projects\IPL_2024_Project\C10_Input_Files\datasets\dim_match_summary.csv')
players = pd.read_csv('D:\Projects\IPL_2024_Project\C10_Input_Files\datasets\dim_players.csv')
batting_summary = pd.read_csv(r'D:\Projects\IPL_2024_Project\C10_Input_Files\datasets\fact_bating_summary.csv')
bowling_summary = pd.read_csv(r'D:\Projects\IPL_2024_Project\C10_Input_Files\datasets\fact_bowling_summary.csv')
                        

In [3]:
print("Match Summary Column:  --> ", match_summary.columns)
print("Players Column:  --> ", players.columns)
print("Batting Summary Column:  --> ", batting_summary.columns)
print("Bowling Summary Column:  --> ", bowling_summary.columns)

Match Summary Column:  -->  Index(['team1', 'team2', 'winner', 'margin', 'matchDate', 'match_id'], dtype='object')
Players Column:  -->  Index(['name', 'team', 'battingStyle', 'bowlingStyle', 'playingRole'], dtype='object')
Batting Summary Column:  -->  Index(['match_id', 'match', 'teamInnings', 'battingPos', 'batsmanName',
       'out/not_out', 'runs', 'balls', '4s', '6s', 'SR'],
      dtype='object')
Bowling Summary Column:  -->  Index(['match_id', 'match', 'bowlingTeam', 'bowlerName', 'overs', 'maiden',
       'runs', 'wickets', 'economy', '0s', '4s', '6s', 'wides', 'noBalls'],
      dtype='object')


In [4]:
print("Match Summary :  --> ", match_summary.shape)
print("Players :  --> ", players.shape)
print("Batting Summary :  --> ", batting_summary.shape)
print("Bowling Summary :  --> ", bowling_summary.shape)

Match Summary :  -->  (206, 6)
Players :  -->  (292, 5)
Batting Summary :  -->  (3268, 11)
Bowling Summary :  -->  (2436, 14)


### Data Cleaning

In [5]:
match_summary.dtypes

team1        object
team2        object
winner       object
margin       object
matchDate    object
match_id     object
dtype: object

In [6]:
batting_summary.dtypes

match_id       object
match          object
teamInnings    object
battingPos      int64
batsmanName    object
out/not_out    object
runs            int64
balls           int64
4s              int64
6s              int64
SR             object
dtype: object

In [7]:
match_summary['year']= (match_summary['matchDate']).str[-4:]
match_summary['year'].unique()

array(['2021', '2022', '2023'], dtype=object)

### Player Stats

#### Create DF having Player name and Runs

In [8]:
playerStat  = pd.DataFrame(players['name'])

In [9]:
batting_stat =  batting_summary.groupby('batsmanName').agg({'runs':'sum','balls':'sum','4s':'sum','6s':'sum'}).reset_index()

In [10]:
#add Runs in playerStat for further Analysis
playerStat['runs'] = playerStat['name'].map(batting_stat.set_index('batsmanName')['runs'])
playerStat['Balls_Faced'] = playerStat['name'].map(batting_stat.set_index('batsmanName')['balls'])
playerStat['4s'] = playerStat['name'].map(batting_stat.set_index('batsmanName')['4s'])
playerStat['6s'] = playerStat['name'].map(batting_stat.set_index('batsmanName')['6s'])
playerStat.head(2)

,name,runs,Balls_Faced,4s,6s
0,RuturajGaikwad,1593.0,1157.0,143.0,67.0
1,FafduPlessis,1831.0,1300.0,169.0,72.0


#### Calculating Batters Played at least 60 balls in Each Season

In [11]:
batting_and_matches = pd.merge(left=batting_summary, right=match_summary, how='inner')

In [12]:
batting_and_matches.columns

Index(['match_id', 'match', 'teamInnings', 'battingPos', 'batsmanName',
       'out/not_out', 'runs', 'balls', '4s', '6s', 'SR', 'team1', 'team2',
       'winner', 'margin', 'matchDate', 'year'],
      dtype='object')

In [13]:
#batting_stat.groupby(['batsmanName','year'])['balls'].sum()

In [14]:
# Pivot table
balls_faced_by_batter = batting_and_matches.pivot_table(index='batsmanName', columns='year', values='balls', aggfunc='sum', fill_value=0)
# Adding total column
balls_faced_by_batter['Total'] = balls_faced_by_batter.sum(axis=1)
balls_faced_by_batter = balls_faced_by_batter.reset_index()
balls_faced_by_batter.index.names=['SRNO']
balls_faced_by_batter.head(2)


year,batsmanName,2021,2022,2023,Total
SRNO,,,,,
0,ABdeVilliers,211,0,0,211
1,AaronFinch,0,61,0,61


In [15]:
#each Season 60 balls faced by batter
eachSeasonBatter =  balls_faced_by_batter[(balls_faced_by_batter['2021'] >= 60) & (balls_faced_by_batter['2022'] >= 60) & (balls_faced_by_batter['2023'] >= 60)].reset_index()
#update values after mapping
playerStat['eachSeasonBatter'] = playerStat['name'].isin(eachSeasonBatter['batsmanName']).apply(lambda x: 'Yes' if x else 'No')
#playerStat.value_counts('eachSeasonBatter')
eachSeasonBatter.head(2)

year,SRNO,batsmanName,2021,2022,2023,Total
0,6,AbhishekSharma,69,320,157,546
1,11,AidenMarkram,119,274,197,590


In [16]:
#batting_summary.head()

#### Batsman Out/Not Out Count required for average

In [17]:
batsman_not_out = batting_summary[batting_summary['out/not_out']=='not_out'].groupby('batsmanName')['out/not_out'].count().reset_index()
batsman_out = batting_summary[batting_summary['out/not_out']=='out'].groupby('batsmanName')['out/not_out'].count().reset_index()
batsman_total_match = batting_summary.groupby('batsmanName')['out/not_out'].count().reset_index()


In [18]:
playerStat['not_out'] = playerStat['name'].map(batsman_not_out.set_index('batsmanName')['out/not_out'])
playerStat['out'] = playerStat['name'].map(batsman_out.set_index('batsmanName')['out/not_out'])
playerStat['Matches_Batted'] = playerStat['name'].map(batsman_total_match.set_index('batsmanName')['out/not_out'])
playerStat.head(2)

,name,runs,Balls_Faced,4s,6s,eachSeasonBatter,not_out,out,Matches_Batted
0,RuturajGaikwad,1593.0,1157.0,143.0,67.0,Yes,3.0,42.0,45.0
1,FafduPlessis,1831.0,1300.0,169.0,72.0,Yes,4.0,42.0,46.0


In [19]:
playerStat['not_out'] = playerStat['not_out'].fillna(0)
playerStat['out'] = playerStat['out'].fillna(0)
playerStat['Matches_Batted'] = playerStat['Matches_Batted'].fillna(0)

#### Calculate Batting Average

In [20]:
playerStat['avgBattingRuns'] = np.round(playerStat['runs']/playerStat['out'],2)
#cleaning NaN values
playerStat.loc[playerStat['out'] == 0, 'avgBattingRuns'] = playerStat['runs']

In [21]:
playerStat.head()

,name,runs,Balls_Faced,4s,6s,eachSeasonBatter,not_out,out,Matches_Batted,avgBattingRuns
0,RuturajGaikwad,1593.0,1157.0,143.0,67.0,Yes,3.0,42.0,45.0,37.93
1,FafduPlessis,1831.0,1300.0,169.0,72.0,Yes,4.0,42.0,46.0,43.60
2,RobinUthappa,345.0,255.0,27.0,19.0,No,0.0,15.0,15.0,23.00
3,MoeenAli,725.0,528.0,67.0,36.0,Yes,4.0,31.0,35.0,23.39
4,ShubmanGill,1851.0,1331.0,186.0,56.0,Yes,4.0,46.0,50.0,40.24


#### Calculate Strike Rate

In [22]:
playerStat['strikeRate'] = np.round(100*playerStat['runs']/playerStat['Balls_Faced'],2)
#cleaning NaN values
#playerStat.loc[playerStat['out'] == 0, 'avgBattingRuns'] = playerStat['runs']

In [23]:
playerStat.head()

,name,runs,Balls_Faced,4s,6s,eachSeasonBatter,not_out,out,Matches_Batted,avgBattingRuns,strikeRate
0,RuturajGaikwad,1593.0,1157.0,143.0,67.0,Yes,3.0,42.0,45.0,37.93,137.68
1,FafduPlessis,1831.0,1300.0,169.0,72.0,Yes,4.0,42.0,46.0,43.60,140.85
2,RobinUthappa,345.0,255.0,27.0,19.0,No,0.0,15.0,15.0,23.00,135.29
3,MoeenAli,725.0,528.0,67.0,36.0,Yes,4.0,31.0,35.0,23.39,137.31
4,ShubmanGill,1851.0,1331.0,186.0,56.0,Yes,4.0,46.0,50.0,40.24,139.07


#### `Calculate Boundary %

In [24]:
playerStat['boundary_percentage'] = round(100*(playerStat['4s']*4 + playerStat['6s']*6)/playerStat['runs'],2)

In [25]:
#playerStat

### Top 10 Batsman

#### Based on Total Runs in Past 3 Years

In [26]:
#top 10 batsman
top_10_batsmen_by_runs = playerStat.nlargest(10, 'runs').reset_index(drop=True)
# Set the index starting from 1
top_10_batsmen_by_runs.index = top_10_batsmen_by_runs.index + 1
top_10_batsmen_by_runs = top_10_batsmen_by_runs[['name','runs']]
print(top_10_batsmen_by_runs)

               name    runs
1       ShubmanGill  1851.0
2      FafduPlessis  1831.0
3    RuturajGaikwad  1593.0
4           KLRahul  1516.0
5        JosButtler  1509.0
6     ShikharDhawan  1392.0
7        ViratKohli  1385.0
8       SanjuSamson  1304.0
9   SuryakumarYadav  1225.0
10     GlennMaxwell  1214.0


#### Based on Average

In [27]:
#Top 10 Batter Based on Past 3 years Average
top_10_batsmen_by_avg = playerStat[playerStat['eachSeasonBatter'] == 'Yes'].nlargest(10, 'avgBattingRuns').reset_index(drop=True)
# Set the index starting from 1
top_10_batsmen_by_avg.index = top_10_batsmen_by_avg.index + 1
top_10_batsmen_by_avg = top_10_batsmen_by_avg[['name','avgBattingRuns']]
print(top_10_batsmen_by_avg)

               name  avgBattingRuns
1           KLRahul           50.53
2      FafduPlessis           43.60
3       DavidMiller           43.20
4        JosButtler           41.92
5    ShimronHetmyer           40.67
6       ShubmanGill           40.24
7     ShikharDhawan           39.77
8    RuturajGaikwad           37.93
9       DavidWarner           37.90
10  SuryakumarYadav           35.00


#### Based on Strike Rate

In [28]:
#Top 10 Batter Based on Past 3 years Strike Rate
top_10_batsmen_by_sr = playerStat[playerStat['eachSeasonBatter'] == 'Yes'].nlargest(10, 'strikeRate').reset_index(drop=True)
# Set the index starting from 1
top_10_batsmen_by_sr.index = top_10_batsmen_by_sr.index + 1
top_10_batsmen_by_sr = top_10_batsmen_by_sr[['name','strikeRate']]
print(top_10_batsmen_by_sr)

               name  strikeRate
1      GlennMaxwell      161.44
2   SuryakumarYadav      160.55
3      AndreRussell      159.19
4    ShimronHetmyer      157.27
5    NicholasPooran      157.11
6       PrithviShaw      153.20
7     DineshKarthik      152.64
8   YashasviJaiswal      152.15
9        JosButtler      146.93
10       ShivamDube      145.95


#### Based on Boundary %

### Top 10 Bowlers

#### Based on Total Wickets in Past 3 Years

In [29]:
bowlers_and_wickets = bowling_summary.groupby('bowlerName')['wickets'].sum().reset_index()

In [30]:
#bowlers_and_wickets.head()

In [31]:
#top 10 bowlers
top_10_bowler = bowlers_and_wickets.nlargest(10, 'wickets').reset_index(drop=True)
# Set the index starting from 1
top_10_bowler.index = top_10_bowler.index + 1
print(top_10_bowler)

           bowlerName  wickets
1       MohammedShami       67
2     YuzvendraChahal       66
3        HarshalPatel       65
4          RashidKhan       63
5           AveshKhan       47
6       ArshdeepSingh       45
7        KagisoRabada       45
8   VarunChakravarthy       44
9       ShardulThakur       43
10         TrentBoult       42


#### Calculating Bowler who has bowled at least 60 balls in each season

#### Based on Average

#### Based on Economy

#### Based on Dot Ball %